В этой части обученная нейросеть проверяется на видеозаписях, которые она не "видела". В основе распознавания видеозаписи лежит лежит модернизированная функция из первой части. Кадр распознается и нормированные данные ключевых точек собираются в словарь для каждого человека, и далее рассчитываются косинус и синус углов относительно вертикали для положения ярук и ног. При накоплении данных за продолжитльность кадров, равной той, на которой обучалась нейросеть, эти данные подаются на вход обученной нейросети. Если нейросеть выдает результат, что id является участником драки, то он обводится красным Bounding boxe и свреху появляяяется янадпись "Fight". На выходе раздела "Запуск функции распознавания в папке" получаем распознанное видео, на котором учатсиники драк обведены карсными рамками.

In [2]:
# Установка OpenCV
!pip install opencv-python-headless

# Установка NumPy
!pip install numpy

# Установка Pandas
!pip install pandas

# Установка TensorFlow
!pip install tensorflow

# Установка Ultralytics для YOLO
!pip install ultralytics

# Установка tqdm для отображения прогресса
!pip install tqdm

# Проверка успешной установки
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import defaultdict
from ultralytics import YOLO
import time
from tqdm import tqdm

print("Все зависимости успешно установлены и импортированы!")


2024-08-20 07:57:05.651573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 07:57:05.666255: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 07:57:05.670756: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-20 07:57:05.682014: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-20 07:57:06.253400: W tensorflow/compiler/tf2

In [3]:
import yt_dlp

# Создаем папку 'test', если она не существует
os.makedirs('test', exist_ok=True)

# Список URL-адресов видео
lectures = [
    "https://youtu.be/YX5ATloUrWM"
]

def download_video(url, index):
    try:
        # Автоматически генерируем имя файла на основе индекса
        title = f'lecture_{index + 1}'
        
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best',  # Загружаем лучшее видео и лучшее аудио и объединяем
            'outtmpl': os.path.join('test', f'{title}.mp4'),  # Сохраняем в папке 'test'
            'merge_output_format': 'mp4',  # Объединяем видео и аудио в формате mp4
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
            print(f"Downloaded {title} as {os.path.join('test', f'{title}.mp4')}")
    except Exception as e:
        print(f"Failed to download video at {url}. Error: {e}")

# Скачивание всех лекций
for index, url in enumerate(lectures):
    download_video(url, index)



[youtube] Extracting URL: https://youtu.be/YX5ATloUrWM
[youtube] YX5ATloUrWM: Downloading webpage
[youtube] YX5ATloUrWM: Downloading ios player API JSON
[youtube] YX5ATloUrWM: Downloading web creator player API JSON
[youtube] YX5ATloUrWM: Downloading m3u8 information
[info] YX5ATloUrWM: Downloading 1 format(s): 244+251
[download] Destination: test/lecture_1.f244.webm
[download] 100% of    6.99MiB in 00:00:02 at 3.04MiB/s     
[download] Destination: test/lecture_1.f251.webm
[download] 100% of  943.70KiB in 00:00:00 at 2.41MiB/s   
[Merger] Merging formats into "test/lecture_1.mp4"
Deleting original file test/lecture_1.f244.webm (pass -k to keep)
Deleting original file test/lecture_1.f251.webm (pass -k to keep)
Downloaded lecture_1 as test/lecture_1.mp4


In [4]:
# Загрузка обученной модели для распознавания

model_path = f'models/best_model_86_3_good.keras'

model = tf.keras.models.load_model(model_path)

I0000 00:00:1724140639.673379  104006 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724140639.677462  104006 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724140639.680272  104006 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724140639.684139  104006 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [5]:
# @title Функция возвращает синус и косинус угла, между линией, образованной координами двух точек, и вертикалью
import math

def calculate_angle(point1, point2):
    # Проверяем, равна ли любая из координат точек нулю
    if 0 in point1 or 0 in point2:
        return 0, 0

    # Вычисляем вектор AB
    vector_AB = (point2[0] - point1[0], point2[1] - point1[1])

    # Вычисляем скалярное произведение векторов AB и (0, -1)
    dot_product = vector_AB[0] * 0 + vector_AB[1] * -1

    # Вычисляем длину вектора AB
    length_AB = math.sqrt(vector_AB[0]**2 + vector_AB[1]**2)

    # Проверяем, не является ли длина вектора нулем (чтобы избежать деления на ноль)
    if length_AB == 0:
        return 0, 0

    # Вычисляем косинус угла между векторами AB и (0, -1)
    cos_angle = dot_product / length_AB

    # Вычисляем синус угла между векторами AB и CD
    if cos_angle < -1:
      cos_angle = -1
    elif cos_angle > 1:
      cos_angle = 1

    sin_angle = math.sqrt(1 - cos_angle**2)

    # Возвращаем косинус и синус угла
    return cos_angle, sin_angle



In [6]:
# @title Функция распознает драку на видео и помещает дерущегося человека в красную рамку.
def extract_predictions_angles(Path, model_fights, count_frame, Out_path = None, model_name = 'yolov8x-pose.pt'):

  _, FileName = os.path.split(Path)
  _, Ext = os.path.splitext(FileName)

  # Создаем путь для сохранения файлов
  if not Out_path:
    Out_path = Path[:-len(Ext)]
  else:
    Out_path = os.path.join(Out_path, FileName[:-len(Ext)])

  # Инициализация модели
  model = YOLO(model_name)

  # Переменные для шрифта номера кадра
  font = cv2.FONT_HERSHEY_COMPLEX

  # Создать объект захвата видео
  video_in = cv2.VideoCapture(Path)

  # Если видео не удалось открыть
  if (video_in.isOpened() == False):
    # Сообщение об ошибке
    print('Не удалось считать кадр из видео!')
    # Освобождаем объект захвата видео
    video_in.release()
    cv2.destroyAllWindows()

  else:
    # Количество кадров в секунду (примерная оценка)
    fps = int(video_in.get(cv2.CAP_PROP_FPS))

    # Всего кадров (примерная оценка)
    frames = int(video_in.get(cv2.CAP_PROP_FRAME_COUNT))

    # Разрешение видео
    width = int(video_in.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_in.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Вывод информации о файле
    print('В видеофайле c частотой кадров: '+str(fps)+' кадр(а,ов)/сек\nразрешением: '+ str(width)+'x'+str(height)+'\nсодержится (но это не точно): '+ str(frames)+' кадр(а,ов).')

    # Создаем объект записи видео для вывода результатов
    video_out = cv2.VideoWriter(Out_path+'_out.avi',cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))

    # Создаем датафрейм pandas для сохранения данных
    Data_df = pd.DataFrame(columns=['Frame', 'Id', 'Coords', 'Keypoints', 'Normalized_KP', 'Conf','KP_Conf'])

    # Создаем словарь для хранения треков
    track_history = defaultdict(lambda: [])

    # Создаем словарь для хранения нормализованных координат человека
    normalized_KP_predict = defaultdict(lambda: [])

    KP_predict = defaultdict(lambda: [])

    # Инициализируем счетчик кадров
    frame_num = 0

    # Прыгаем к нужному кадру
    if not frame_num == 0:
      if 0 < frame_num < frames:
        video_in.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
      else:
        print('Стартовый кадр задан не корректно и будет сброшен на 0!')
        frame_num = 0

    # Считываем кадр
    is_read, frame = video_in.read()
    with tqdm(total=frames) as pbar:
      while is_read:
        # Обрабатываем кадр
        results = model.track(frame, persist=True, verbose=False)
        try:
          # Извлекаем BBoxы и ID треков
          boxes_xywh = results[0].boxes.xywh.cpu().tolist()
          boxes_xyxy = results[0].boxes.xyxy.cpu().tolist()
          track_ids = results[0].boxes.id.int().cpu().tolist()
          confs = results[0].boxes.conf.cpu().tolist()

          # Извлекаем относительные положени ключевых точек и уверенность сети в их определении
          all_keypoints = results[0].keypoints.xy.cpu().tolist()
          all_kp_confs = results[0].keypoints.conf.cpu().tolist()

          for box_xywh, box_xyxy, track_id, conf, keypoints, kp_confs in zip(boxes_xywh, boxes_xyxy, track_ids, confs, all_keypoints, all_kp_confs):

              # Определяем координаты рамки
              x, y, w, h = box_xywh
              x_min, y_min, x_max, y_max = box_xyxy

              # Вычисляем нормированные значения keypoints
              delta = [x_min, y_min]
              scaler = [x_max - x_min, y_max - y_min]

              normalized_KP = []
              # Вариант сохранения нулевых normalized_KP для нулевых значений keypoints
              for coord in keypoints:
                  if coord == [0.0, 0.0]:
                      normalized_KP.append([-1.0, -1.0])
                  else:
                      normalized_coord = ((np.asarray(coord) - np.asarray(delta)) / np.asarray(scaler)).tolist()
                      normalized_KP.append(normalized_coord)

              # Добавляем данные в датафрейм
              Data_df.loc[len(Data_df.index)] = [frame_num, track_id, box_xyxy, keypoints, normalized_KP, conf, kp_confs]

              # Добавление normalized_KP в словарь
              KP_predict = normalized_KP_predict[track_id]
              KP_predict.append(keypoints)
              if len(KP_predict) == count_frame:
                angles_summ = []
                for item in KP_predict:

                  angles = []
                  # плечо
                  angles += [calculate_angle(item[5], item[7])]
                  angles += [calculate_angle(item[6], item[8])]
                  # предплечье
                  angles += [calculate_angle(item[7], item[9])]
                  angles += [calculate_angle(item[8], item[10])]
                  # бедро
                  angles += [calculate_angle(item[11], item[13])]
                  angles += [calculate_angle(item[12], item[14])]
                  # гоень
                  angles += [calculate_angle(item[13], item[15])]
                  angles += [calculate_angle(item[14], item[16])]
                  angles_summ.append(angles)

                KP_predict = np.array(angles_summ)
                KP_predict = np.array(KP_predict).reshape(1, count_frame, 8, 2)
                prediction = model_fights.predict(KP_predict, verbose=0)
                normalized_KP_predict[track_id].pop(0)

                if prediction[0][0] > 0.90:
                      # Преобразуем координаты в целые числа
                  x, y, w, h = int(x), int(y), int(w), int(h)
                  x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)

                  # Наносим рамку на кадр
                  cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 0, 255), 3)  # Цвет рамки: красный, толщина: 3 пикселя

                  # Добавляем надпись на кадр над рамкой
                  text = "fight"  # Ваш текст
                  org = (x_min, y_min - 10)  # Координаты начала текста (над рамкой)
                  font = cv2.FONT_HERSHEY_SIMPLEX  # Шрифт
                  font_scale = 1  # Масштаб шрифта
                  font_color = (0, 0, 255)  # Цвет текста: красный
                  thickness = 3  # Толщина линии
                  cv2.putText(frame, text, org, font, font_scale, font_color, thickness)

          # Сохраняем обработанный кадр в видео
          video_out.write(frame)
          # Обновление прогресс-бара

        except:
          # Сохраняем обработанный кадр в видео
          video_out.write(frame)
        # Убеждаемся, что прогресс-бар достиг 100%

        # Считываем следующий кадр
        is_read, frame = video_in.read()
        pbar.update(1)
        # Увеличиваем счетчик кадров
        frame_num += 1

    pbar.close()
    # Освобождаем объекты видео
    video_in.release()
    video_out.release()
    cv2.destroyAllWindows()


    return [FileName, Out_path+'_out.avi']

In [7]:
# @title Вспомогательные функции

# Контекстный менеджер для измерения времени операций
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        # Расчет времени выполнения
        result_time = time.time()-self.t
        hour = int(result_time//3600)
        min = int(result_time//60)-hour*60
        sec = int(round(result_time%60))
        msec = round(1000*result_time%60)

        if hour > 0:
          print('Время обработки: ' + str(hour)+' час. ' + str(min)+' мин.')
        elif min > 0:
          print('Время обработки: ' + str(min)+' мин. ' + str(sec)+' сек.')
        elif sec > 0:
          print('Время обработки: ' + str(sec)+' сек.')
        else:
          print('Время обработки: ' + str(msec)+' мс.')

In [8]:
# @title Запуск функции распознавания в папке
with timex():
  # Путь к каталогу с видео для обработки
  source = 'test/'

  # Путь для сохранения результатов обработки
  destination = 'result/'

  # Список расширений файлов (на случай если в папке еще что-то есть)
  ext_list = ['.mp4','.avi','.mov']

  # Список для хранения имен файлов
  Files_List = []

  # Список для хранения имен файлов
  Out_Video_List = []

  # Список для хранения имен файлов
  Out_Pandas_List = []

  # кол-во кадров идущих подряд, используемых при обучении
  count_frame = model.input_shape[1]

  # Создание каталога для сохранения при его отсутствии
  if not os.path.exists(destination):
    os.mkdir(destination)

  if os.path.isfile(destination):
    print('Существует файл идентичный имени каталога!\nУдалите/переименуйте файл или измените путь для сохранения.')
  else:
    # Запуск обработки
    for file_name in os.listdir(source):
      _, ext = os.path.splitext(file_name)
      if ext in ext_list:
        with timex():
          print(f'\nНачало обработки файла {file_name}.')
          FileName, Out_Video = extract_predictions_angles(os.path.join(source,file_name), model, count_frame, destination, 'yolov8x-pose.pt')
          Files_List.append(FileName)
          Out_Video_List.append(Out_Video)
          print(f'\nФайл обработан.\nВидео сохранено в: {Out_Video}')

    print(f'\nОбработка закончена. Всего обработано {len(Files_List)} файл(а,ов).')


Начало обработки файла lecture_1.mp4.
В видеофайле c частотой кадров: 29 кадр(а,ов)/сек
разрешением: 480x848
содержится (но это не точно): 2072 кадр(а,ов).


  0%|          | 1/2072 [00:00<34:27,  1.00it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1724140650.946081  104111 service.cc:146] XLA service 0x7fe228004100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724140650.946109  104111 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-08-20 07:57:30.953632: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-20 07:57:30.976246: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
I0000 00:00:1724140651.111128  104111 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
 13%|█▎        | 271/2072 [00:19<02:54, 10.31it/s]

# Подведение итогов
В процессе работы были получены два варианта датасета. Изначально был создан дата сет, с различными отрицательными нормированными значениями координат ключевых точек (при распознавании YOLO-pose эти координаты были равны 0, но при нормировании они принимали отрицательное значение, причем разное для каждого кадра). Нейросеть, обученная на таком дата сете выдавала неудовлетворительный результат, и точность поднять с помощью перебора архитектуры не удавалось. Поэтому было принято решение изменить функцию extract_predictions так, чтобы всем нормируемым значениям координат ключевых точек, которые не были распознаны YOLO-pose присваивать значения "-1". Видеозаписи были заново распознаны и был получен дата сет второй версии. На нем удалось поднять точность до 82%. Визуальная проверка на видеозаписях, которые не "видела" нейросеть была уже намного лучше, но так же точность не уадавалось поднять выше. Для увеличения точности было принято решение взять углы рук и ног человека относительно вертикали (8 углов для каждогго человека - плечо, предплечье, бедро и голень). Был увеличен датасет и был заново распознан. С помощью нового генератора был создан датасет с углами и произведен поиск архитектуры с помощью autokeras. Было сделано более 600 trials, была подобрана очень простая нейросеть из слоев dense. ДЛя эксперимента было проведено еще около 400 trials  в autokeras с блоками ConvBlock, RNNBlock и ResNetBlock, изменив при этом размерность датасета, чтобы данные можно было рассматривать как последовательность временных рядов. Но точность осталась на том же уровне, но с более сложной архитектурой. На данный момент выбрана самая простая и быстрая нейросет, которая обладает точностью 90.5% на валидационной выборке. На видеозаписях, которые нейросеть не видела получились хорошие результаты.